# 📦 Guía Completa: Archivos Comprimidos en Python

Aprende cómo trabajar con archivos comprimidos (TAR.GZ, ZIP y RAR) usando Python.
- Crear archivos comprimidos
- Extraer contenido
- Listar archivos sin extraer
- Procesar datos en memoria
- Comparativa de formatos

## 1️⃣ Importar Librerías Necesarias

Para trabajar con archivos comprimidos en Python, necesitamos las siguientes librerías:

| Librería | Formato | Estado | Notas |
|----------|---------|--------|-------|
| `tarfile` | TAR, TAR.GZ, TAR.BZ2 | ✅ Integrada | Viene con Python |
| `zipfile` | ZIP | ✅ Integrada | Viene con Python |
| `gzip`, `bz2` | GZ, BZ2 | ✅ Integrada | Complementos para tarfile |
| `rarfile` | RAR | 📦 Externa | Requiere instalación |

**Instalación de rarfile:**
```bash
pip install rarfile
```

In [ ]:
import tarfile
import zipfile
import gzip
import bz2
import os
from pathlib import Path
from datetime import datetime

# Intentar importar rarfile (se instalará después si no está disponible)
RARFILE_AVAILABLE = False
try:
    import rarfile
    RARFILE_AVAILABLE = True
    print("✅ rarfile está disponible")
except ImportError:
    print("⚠️  rarfile no está instalado.")
    print("    Comando manual: pip install rarfile")

print("✅ Librerías base importadas correctamente")
print(f"   rarfile disponible: {RARFILE_AVAILABLE}")

## 2️⃣ Comparativa de Formatos de Archivos Comprimidos

### 📊 Tabla comparativa

| Característica | TAR.GZ | ZIP | RAR |
|----------------|--------|-----|-----|
| **Múltiples archivos** | ✅ SÍ | ✅ SÍ | ✅ SÍ |
| **Preserva permisos** | ✅ SÍ | ❌ NO | ✅ SÍ |
| **Compresión** | GZIP | DEFLATE | Propietaria |
| **Multiplataforma** | ✅ SÍ | ✅ SÍ | ✅ SÍ |
| **Soporte nativo Windows** | ❌ NO* | ✅ SÍ | ❌ NO |
| **Soporte nativo Linux** | ✅ SÍ | ✅ SÍ | ❌ NO |
| **Mejor compresión** | TAR.BZ2 | - | Excelente |
| **Velocidad** | Rápida | Muy rápida | Lenta |

### 🎯 Cuándo usar cada formato

- **TAR.GZ**: Sistema Unix/Linux, máxima compatibilidad, preservar permisos
- **ZIP**: Windows, máxima compatibilidad, no necesita herramientas especiales
- **RAR**: Máxima compresión, archivos muy grandes, backup profesional

---

# 📁 TRABAJAR CON ARCHIVOS TAR.GZ

## 3️⃣ Crear Archivos TAR.GZ

Los archivos TAR.GZ son perfectos para:
- Empaquetar y comprimir múltiples archivos
- Preservar la estructura de carpetas
- Preservar permisos de archivo (importante en Linux)
- Máxima compatibilidad en sistemas Unix/Linux

In [ ]:
# Crear archivos de prueba
os.makedirs("datos_prueba", exist_ok=True)

# Crear algunos archivos de ejemplo
with open("datos_prueba/archivo1.txt", "w") as f:
    f.write("Este es el contenido del archivo 1\n" * 100)

with open("datos_prueba/archivo2.txt", "w") as f:
    f.write("Este es el contenido del archivo 2\n" * 50)

os.makedirs("datos_prueba/subdir", exist_ok=True)
with open("datos_prueba/subdir/archivo3.txt", "w") as f:
    f.write("Este es el contenido del archivo 3\n" * 25)

print("✅ Archivos de prueba creados en la carpeta 'datos_prueba'")

In [ ]:
def crear_tar_gz(directorio, archivo_salida):
    """Crea un archivo TAR.GZ a partir de un directorio.
    
    Args:
        directorio (str): Ruta del directorio a comprimir
        archivo_salida (str): Nombre del archivo TAR.GZ de salida
    """
    with tarfile.open(archivo_salida, "w:gz") as tar:
        # Agregar todo el directorio
        tar.add(directorio, arcname=os.path.basename(directorio))
        print(f"✅ Archivo creado: {archivo_salida}")
        print(f"📊 Tamaño: {os.path.getsize(archivo_salida) / 1024:.2f} KB")

# Crear archivo TAR.GZ
crear_tar_gz("datos_prueba", "backup.tar.gz")

## 4️⃣ Listar Contenido de TAR.GZ (sin extraer)

In [ ]:
def listar_tar_gz(archivo_tar):
    """Lista el contenido de un archivo TAR.GZ sin extraerlo.
    
    Args:
        archivo_tar (str): Ruta del archivo TAR.GZ
    """
    print(f"📦 Contenido de {archivo_tar}:\n")
    print(f"{'TIPO':<6} {'TAMAÑO':<12} {'RUTA':<50}")
    print("-" * 70)
    
    with tarfile.open(archivo_tar, "r:gz") as tar:
        tamaño_total = 0
        for miembro in tar.getmembers():
            tipo = "DIR" if miembro.isdir() else "FILE"
            tamaño = f"{miembro.size} B"
            print(f"{tipo:<6} {tamaño:<12} {miembro.name:<50}")
            tamaño_total += miembro.size
        
        print("-" * 70)
        print(f"📊 Total: {tamaño_total / 1024:.2f} KB")

# Listar contenido
listar_tar_gz("backup.tar.gz")

## 5️⃣ Extraer Archivos TAR.GZ

In [ ]:
def extraer_tar_gz(archivo_tar, directorio_destino="."):
    """Extrae un archivo TAR.GZ a un directorio específico.
    
    Args:
        archivo_tar (str): Ruta del archivo TAR.GZ
        directorio_destino (str): Directorio donde extraer (por defecto actual)
    """
    os.makedirs(directorio_destino, exist_ok=True)
    
    with tarfile.open(archivo_tar, "r:gz") as tar:
        tar.extractall(path=directorio_destino)
        print(f"✅ Extractado en: {directorio_destino}")
        print(f"📁 Archivos extraídos: {len(tar.getmembers())}")

# Extraer a una carpeta temporal
extraer_tar_gz("backup.tar.gz", "backup_extraido")

## 6️⃣ Procesar Archivos TAR.GZ sin Extraer a Disco

Una ventaja clave: **leer archivos en memoria sin extraer a disco**

In [ ]:
def procesar_tar_gz_en_memoria(archivo_tar, extension=".txt"):
    """Procesa archivos dentro de TAR.GZ sin extraer a disco.
    
    Args:
        archivo_tar (str): Ruta del archivo TAR.GZ
        extension (str): Extensión de archivo a procesar
    """
    print(f"📖 Leyendo {archivo_tar} en memoria...\n")
    
    with tarfile.open(archivo_tar, "r:gz") as tar:
        for miembro in tar.getmembers():
            if miembro.isfile() and miembro.name.endswith(extension):
                # Extraer archivo en memoria (sin guardarlo en disco)
                f = tar.extractfile(miembro)
                contenido = f.read().decode('utf-8')
                
                print(f"📄 {miembro.name}")
                print(f"   Tamaño: {miembro.size} bytes")
                print(f"   Primeros 100 caracteres: {contenido[:100]}...")
                print()

# Procesar archivos TAR.GZ en memoria
procesar_tar_gz_en_memoria("backup.tar.gz", ".txt")

---

# 📦 TRABAJAR CON ARCHIVOS ZIP

## 7️⃣ Crear Archivos ZIP

ZIP es el formato más compatible en Windows y es muy popular para distribución.

In [ ]:
def crear_zip(directorio, archivo_salida, compresion=zipfile.ZIP_DEFLATED):
    """Crea un archivo ZIP a partir de un directorio.
    
    Args:
        directorio (str): Ruta del directorio a comprimir
        archivo_salida (str): Nombre del archivo ZIP de salida
        compresion: Tipo de compresión (ZIP_DEFLATED, ZIP_STORED, etc)
    """
    with zipfile.ZipFile(archivo_salida, 'w', compresion) as zipf:
        for root, dirs, files in os.walk(directorio):
            for file in files:
                file_path = os.path.join(root, file)
                # Agregar archivo manteniendo estructura de carpetas
                arcname = os.path.relpath(file_path, os.path.dirname(directorio))
                zipf.write(file_path, arcname)
        
        print(f"✅ Archivo ZIP creado: {archivo_salida}")
        print(f"📊 Tamaño: {os.path.getsize(archivo_salida) / 1024:.2f} KB")
        print(f"📁 Archivos incluidos: {len(zipf.namelist())}")

# Crear archivo ZIP
crear_zip("datos_prueba", "backup.zip")

## 8️⃣ Listar Contenido de ZIP (sin extraer)

In [ ]:
def listar_zip(archivo_zip):
    """Lista el contenido de un archivo ZIP sin extraerlo.
    
    Args:
        archivo_zip (str): Ruta del archivo ZIP
    """
    print(f"📦 Contenido de {archivo_zip}:\n")
    print(f"{'TAMAÑO ORIGINAL':<15} {'COMPRIMIDO':<15} {'RUTA':<50}")
    print("-" * 80)
    
    with zipfile.ZipFile(archivo_zip, 'r') as zipf:
        tamaño_total = 0
        comprimido_total = 0
        
        for info in zipf.infolist():
            original = f"{info.file_size} B"
            comprimido = f"{info.compress_size} B"
            print(f"{original:<15} {comprimido:<15} {info.filename:<50}")
            tamaño_total += info.file_size
            comprimido_total += info.compress_size
        
        print("-" * 80)
        ratio = (1 - comprimido_total / tamaño_total) * 100 if tamaño_total > 0 else 0
        print(f"📊 Total: {tamaño_total / 1024:.2f} KB → {comprimido_total / 1024:.2f} KB")
        print(f"📉 Compresión: {ratio:.1f}%")

# Listar contenido ZIP
listar_zip("backup.zip")

## 9️⃣ Extraer Archivos ZIP

In [ ]:
def extraer_zip(archivo_zip, directorio_destino="."):
    """Extrae un archivo ZIP a un directorio específico.
    
    Args:
        archivo_zip (str): Ruta del archivo ZIP
        directorio_destino (str): Directorio donde extraer
    """
    os.makedirs(directorio_destino, exist_ok=True)
    
    with zipfile.ZipFile(archivo_zip, 'r') as zipf:
        zipf.extractall(path=directorio_destino)
        print(f"✅ Extractado en: {directorio_destino}")
        print(f"📁 Archivos extraídos: {len(zipf.namelist())}")

# Extraer ZIP
extraer_zip("backup.zip", "backup_zip_extraido")

## 🔟 Procesar Archivos ZIP sin Extraer a Disco

In [ ]:
def procesar_zip_en_memoria(archivo_zip, extension=".txt"):
    """Procesa archivos dentro de ZIP sin extraer a disco.
    
    Args:
        archivo_zip (str): Ruta del archivo ZIP
        extension (str): Extensión de archivo a procesar
    """
    print(f"📖 Leyendo {archivo_zip} en memoria...\n")
    
    with zipfile.ZipFile(archivo_zip, 'r') as zipf:
        for info in zipf.infolist():
            if not info.is_dir() and info.filename.endswith(extension):
                # Leer archivo en memoria
                contenido = zipf.read(info.filename).decode('utf-8')
                
                print(f"📄 {info.filename}")
                print(f"   Tamaño: {info.file_size} bytes")
                print(f"   Primeros 100 caracteres: {contenido[:100]}...")
                print()

# Procesar ZIP en memoria
procesar_zip_en_memoria("backup.zip", ".txt")

---

# 📦 TRABAJAR CON ARCHIVOS RAR

## 1️⃣1️⃣ Introducción a RAR

RAR es un formato propietario con excelente compresión, pero requiere bibliotecas especiales.

**⚠️ Nota importante:**
- Para **crear** archivos RAR necesitas WinRAR (software propietario)
- Python puede **leer y extraer** archivos RAR con `rarfile`
- `rarfile` es solo un wrapper que usa `unrar` en el sistema

In [ ]:
# Verificar e instalar rarfile automáticamente (opcional)
# Descomenta la siguiente línea si deseas instalarlo automáticamente

if not RARFILE_AVAILABLE:
    print("\n" + "="*70)
    print("🔧 rarfile NO está disponible")
    print("="*70)
    print("\n✅ Instalación manual (RECOMENDADO):")
    print("   pip install rarfile")
    print("\n✅ Instalación automática (opcional):")
    print("   Descomenta la siguiente celda si deseas instalar automáticamente")
    print("\n" + "="*70)
else:
    print("✅ rarfile ya está instalado y disponible")

In [ ]:
# ⚡ INSTALACIÓN AUTOMÁTICA (OPCIONAL)
# Esta celda solo se ejecuta si rarfile NO está disponible

if RARFILE_AVAILABLE:
    print("✅ rarfile ya está disponible, no es necesario instalar")
else:
    import subprocess
    import sys
    
    print("🔧 Iniciando instalación automática de rarfile...\n")
    
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "rarfile"])
        print("✅ rarfile instalado exitosamente\n")
        
        # Reimportar rarfile después de instalarlo
        try:
            import rarfile
            RARFILE_AVAILABLE = True
            print("✅ rarfile cargado correctamente")
            print(f"✅ RARFILE_AVAILABLE = {RARFILE_AVAILABLE}")
        except ImportError:
            print("❌ No se pudo importar rarfile después de instalar")
            print("   Asegúrate de tener unrar instalado en tu sistema")
            RARFILE_AVAILABLE = False
    
    except subprocess.CalledProcessError as e:
        print(f"❌ Error durante la instalación: {e}")
        print("\nIntenta instalar manualmente con:")
        print("   pip install rarfile")
        RARFILE_AVAILABLE = False
    except Exception as e:
        print(f"❌ Error inesperado: {e}")
        RARFILE_AVAILABLE = False

### Instalación Manual de rarfile

Si rarfile no está disponible, instálalo manualmente con uno de estos comandos:

**Opción 1: Con pip (recomendado)**
```bash
pip install rarfile
```

**Opción 2: Con conda (si usas Anaconda):**
```bash
conda install -c conda-forge rarfile
```

**⚠️ Nota importante:** 
- `rarfile` solo proporciona el wrapper de Python
- También necesitas que **`unrar`** esté instalado en tu sistema:
  - **Windows**: Descarga WinRAR (incluye unrar)
  - **Linux**: `sudo apt-get install unrar` (Debian/Ubuntu) o equivalente
  - **macOS**: `brew install unrar` (si tienes Homebrew)

Después de instalar, ejecuta la siguiente celda para verificar la instalación.

## 1️⃣2️⃣ Listar Contenido de RAR (sin extraer)

In [ ]:
def listar_rar(archivo_rar):
    """Lista el contenido de un archivo RAR sin extraerlo.
    
    Args:
        archivo_rar (str): Ruta del archivo RAR
    """
    if not RARFILE_AVAILABLE:
        print("❌ rarfile no está disponible")
        return
    
    try:
        print(f"📦 Contenido de {archivo_rar}:\n")
        print(f"{'TAMAÑO':<15} {'FECHA':<20} {'RUTA':<50}")
        print("-" * 85)
        
        with rarfile.RarFile(archivo_rar) as rar:
            tamaño_total = 0
            
            for info in rar.infolist():
                tamaño = f"{info.file_size} B"
                fecha = info.date_time.strftime("%Y-%m-%d %H:%M:%S")
                print(f"{tamaño:<15} {fecha:<20} {info.filename:<50}")
                tamaño_total += info.file_size
            
            print("-" * 85)
            print(f"📊 Total: {tamaño_total / 1024:.2f} KB")
    
    except rarfile.Error as e:
        print(f"❌ Error: {e}")
        print("⚠️  Asegúrate de que unrar esté instalado en el sistema")

# Ejemplo (requiere un archivo RAR existente)
print("⚠️  Función listar_rar() definida")
print("    Para probarla, necesitas un archivo .rar")
print("    Ejemplo: listar_rar('archivo.rar')")

## 1️⃣3️⃣ Extraer Archivos RAR

In [ ]:
def extraer_rar(archivo_rar, directorio_destino="."):
    """Extrae un archivo RAR a un directorio específico.
    
    Args:
        archivo_rar (str): Ruta del archivo RAR
        directorio_destino (str): Directorio donde extraer
    """
    if not RARFILE_AVAILABLE:
        print("❌ rarfile no está disponible")
        return
    
    try:
        os.makedirs(directorio_destino, exist_ok=True)
        
        with rarfile.RarFile(archivo_rar) as rar:
            rar.extractall(path=directorio_destino)
            print(f"✅ Extractado en: {directorio_destino}")
            print(f"📁 Archivos extraídos: {len(rar.infolist())}")
    
    except rarfile.Error as e:
        print(f"❌ Error: {e}")
        print("⚠️  Asegúrate de que unrar esté instalado en el sistema")

print("✅ Función extraer_rar() definida")

---

# 📊 COMPARATIVA Y CASOS DE USO

## 1️⃣4️⃣ Tabla Comparativa Completa

| Aspecto | TAR.GZ | ZIP | RAR |
|--------|--------|-----|-----|
| **Múltiples archivos** | ✅ | ✅ | ✅ |
| **Preserva permisos Unix** | ✅ | ❌ | ✅ |
| **Preserva enlaces simbólicos** | ✅ | ❌ | ❌ |
| **Compatibilidad nativa Windows** | ❌ | ✅ | ❌ |
| **Compatibilidad nativa Linux** | ✅ | ✅ | ❌ |
| **Librería Python nativa** | ✅ | ✅ | ❌ |
| **Crear desde Python** | ✅ | ✅ | ❌* |
| **Velocidad compresión** | ⚡⚡⚡ | ⚡⚡⚡ | ⚡ |
| **Ratio compresión** | ⭐⭐ | ⭐⭐ | ⭐⭐⭐ |
| **Tamaño de archivo típico** | Medio | Medio | Pequeño |
| **Ideal para** | Linux/Backup | Windows/Distribución | Máxima compresión |

*RAR requiere WinRAR (software propietario)

## 1️⃣5️⃣ Cuándo usar cada formato

### 🐧 Usa TAR.GZ cuando:
- Trabajas en sistemas Linux/Unix
- Necesitas preservar permisos de archivo
- Quieres máxima compatibilidad en servidores
- Necesitas empaquetar y comprimir juntos
- Usas herramientas de backup profesionales

```bash
# Ejemplo en terminal
tar -czf backup.tar.gz directorio/
tar -xzf backup.tar.gz
```

### 🪟 Usa ZIP cuando:
- Trabajas principalmente en Windows
- Necesitas máxima compatibilidad universal
- Quieres que el usuario final abra sin herramientas
- Necesitas velocidad de compresión/descompresión
- Distribuyes software a usuarios variados

```bash
# Ejemplo en terminal
zip -r archivo.zip directorio/
unzip archivo.zip
```

### 📦 Usa RAR cuando:
- Necesitas máxima compresión de datos
- Tienes archivos muy grandes
- Usas software especializado (como en empresas)
- Quieres soporte para multi-volúmenes
- Necesitas seguridad adicional (encriptación)

---

# 🎯 EJEMPLOS PRÁCTICOS

## 1️⃣6️⃣ Utilidad Universal: Trabajar con Cualquier Formato

In [ ]:
def universal_listar(archivo):
    """Abre cualquier formato y lista su contenido automáticamente.
    
    Detecta el formato automáticamente por la extensión.
    
    Args:
        archivo (str): Ruta del archivo comprimido
    """
    extension = archivo.lower().split('.')[-1]
    
    if extension == 'zip':
        listar_zip(archivo)
    elif extension in ['gz', 'tar']:
        listar_tar_gz(archivo)
    elif extension == 'rar':
        listar_rar(archivo)
    else:
        print(f"❌ Formato no reconocido: {extension}")
        print("   Formatos soportados: zip, gz, tar, rar")

def universal_extraer(archivo, directorio_destino="."):
    """Extrae cualquier formato automáticamente.
    
    Args:
        archivo (str): Ruta del archivo comprimido
        directorio_destino (str): Donde extraer
    """
    extension = archivo.lower().split('.')[-1]
    
    if extension == 'zip':
        extraer_zip(archivo, directorio_destino)
    elif extension in ['gz', 'tar']:
        extraer_tar_gz(archivo, directorio_destino)
    elif extension == 'rar':
        extraer_rar(archivo, directorio_destino)
    else:
        print(f"❌ Formato no reconocido: {extension}")

# Probar función universal
print("✅ Funciones universales definidas:")
print("   - universal_listar(archivo)")
print("   - universal_extraer(archivo, destino)")
print("\nEjemplo:")
print("   universal_listar('backup.zip')")
print("   universal_extraer('backup.tar.gz', 'extraido')")

In [ ]:
# Probar funciones universales
print("=" * 70)
print("LISTAR CONTENIDO")
print("=" * 70)
universal_listar("backup.zip")

print("\n" + "=" * 70)
print("COMPARAR TAMAÑOS")
print("=" * 70)

print("\n📊 Archivo original (directorio):")
total = 0
for root, dirs, files in os.walk("datos_prueba"):
    for file in files:
        size = os.path.getsize(os.path.join(root, file))
        total += size
print(f"   Tamaño: {total / 1024:.2f} KB")

print("\n📦 Archivo TAR.GZ:")
size_tar = os.path.getsize("backup.tar.gz")
print(f"   Tamaño: {size_tar / 1024:.2f} KB")
print(f"   Compresión: {(1 - size_tar/total)*100:.1f}%")

print("\n📦 Archivo ZIP:")
size_zip = os.path.getsize("backup.zip")
print(f"   Tamaño: {size_zip / 1024:.2f} KB")
print(f"   Compresión: {(1 - size_zip/total)*100:.1f}%")

---

## 1️⃣7️⃣ Crear Utilidad CLI (Línea de Comandos)

In [ ]:
#!/usr/bin/env python3
"""
Utilidad universal para trabajar con archivos comprimidos.
Uso: python archive_tool.py <comando> <archivo> [opciones]
"""

import sys
import argparse

def main():
    parser = argparse.ArgumentParser(
        description="Utilidad universal para archivos comprimidos (ZIP, TAR.GZ, RAR)"
    )
    
    subparsers = parser.add_subparsers(dest='comando', help='Comando a ejecutar')
    
    # Comando: listar
    list_parser = subparsers.add_parser('list', help='Listar contenido del archivo')
    list_parser.add_argument('archivo', help='Archivo comprimido')
    
    # Comando: extraer
    extract_parser = subparsers.add_parser('extract', help='Extraer archivo')
    extract_parser.add_argument('archivo', help='Archivo comprimido')
    extract_parser.add_argument('-d', '--dest', default='.', help='Directorio destino')
    
    # Comando: crear
    create_parser = subparsers.add_parser('create', help='Crear archivo comprimido')
    create_parser.add_argument('output', help='Archivo de salida')
    create_parser.add_argument('source', help='Directorio o archivo a comprimir')
    create_parser.add_argument('-f', '--format', choices=['zip', 'tar.gz'], 
                               default='zip', help='Formato (zip o tar.gz)')
    
    args = parser.parse_args()
    
    if args.comando == 'list':
        universal_listar(args.archivo)
    elif args.comando == 'extract':
        universal_extraer(args.archivo, args.dest)
    elif args.comando == 'create':
        if args.format == 'zip':
            crear_zip(args.source, args.output)
        else:
            crear_tar_gz(args.source, args.output)
    else:
        parser.print_help()

# Para usar desde línea de comandos:
# python archive_tool.py list archivo.zip
# python archive_tool.py extract archivo.tar.gz -d destino/
# python archive_tool.py create backup.zip mi_carpeta/

print("✅ Script CLI definido")
print("\nEjemplos de uso desde terminal:")
print("  python archivo.py list backup.zip")
print("  python archivo.py extract backup.tar.gz -d extraido/")
print("  python archivo.py create backup.zip datos_prueba/")

---

## 1️⃣8️⃣ Referencia Rápida

### Operaciones básicas

```python
# TAR.GZ
import tarfile
tar = tarfile.open('archivo.tar.gz', 'r:gz')
tar.extractall()
tar.close()

# ZIP
import zipfile
zf = zipfile.ZipFile('archivo.zip', 'r')
zf.extractall()
zf.close()

# RAR (requiere rarfile)
import rarfile
rf = rarfile.RarFile('archivo.rar')
rf.extractall()
rf.close()
```

### Crear archivos

```python
# TAR.GZ
tar = tarfile.open('backup.tar.gz', 'w:gz')
tar.add('directorio')
tar.close()

# ZIP
zf = zipfile.ZipFile('backup.zip', 'w')
zf.write('archivo.txt')
zf.close()
```

### Listar contenido

```python
# TAR.GZ
tar = tarfile.open('archivo.tar.gz')
for member in tar.getmembers():
    print(member.name)

# ZIP
zf = zipfile.ZipFile('archivo.zip')
for info in zf.infolist():
    print(info.filename)

# RAR
rf = rarfile.RarFile('archivo.rar')
for info in rf.infolist():
    print(info.filename)
```

### Procesar en memoria

```python
# TAR.GZ
with tarfile.open('archivo.tar.gz') as tar:
    for member in tar.getmembers():
        if member.isfile():
            f = tar.extractfile(member)
            contenido = f.read()

# ZIP
with zipfile.ZipFile('archivo.zip') as zf:
    for filename in zf.namelist():
        contenido = zf.read(filename)

# RAR
with rarfile.RarFile('archivo.rar') as rf:
    for info in rf.infolist():
        contenido = rf.read(info.filename)
```

---

## 🎓 Resumen y Conclusiones

### ✅ Lo que aprendiste

1. **TAR.GZ**: El estándar en Linux, preserva permisos y es muy compatible
2. **ZIP**: El más universal, especialmente en Windows y para distribución
3. **RAR**: Máxima compresión pero requiere herramientas especiales
4. Cómo **listar**, **crear**, y **extraer** archivos sin ocupar espacio en disco
5. Procesar archivos **en memoria** para máxima eficiencia
6. Crear utilidades **universales** que funcionan con cualquier formato

### 🚀 Próximos pasos

- Integra estas funciones en tus proyectos
- Crea herramientas de backup automatizadas
- Implementa descarga y extracción de archivos en tu aplicación
- Comprime logs antiguos en tus sistemas
- Automatiza tareas de administración de datos

### 📚 Recursos útiles

```bash
# Ver documentación en terminal
python -m pydoc tarfile
python -m pydoc zipfile

# Instalaciones necesarias
pip install rarfile  # Para soporte RAR
```

### 💡 Tips importantes

- Siempre usa context managers (`with`) para manejo seguro de archivos
- TAR.GZ es mejor para preservar estructura Unix
- ZIP es mejor para distribución multiplatforma
- RAR ofrece mejor compresión pero con herramientas limitadas en Python
- Procesa archivos en memoria cuando sea posible para ahorrar espacio